**This notebook contains a script that determines whether students are eligible for the SETA NQF level 5 Qualification**

**Import Libraries**

In [5]:
import pandas as pd
import numpy as np
import gspread
import gspread_dataframe as gd
from fuzzywuzzy import process

**Authenticate Gspread**

In [6]:
gc = gspread.oauth()

**Create Worksheets**

In [7]:
#cohort 2020 data worksheet
cohort_2020_data_worksheet = gc.open("Cohort 2020 Data").worksheet('Cohort 2020 Data')

**Import Data into Dataframes**

In [8]:
#import cohort data
cohort_df = gd.get_as_dataframe(cohort_2020_data_worksheet)

#import emis national data 
national_df = pd.read_excel('../national.xlsx')

#import national area codes
areacode_df = pd.read_csv('../temp_area.csv')

#import city to province map
province_df = pd.read_csv("../new_province.csv")

#import municipality data
municipality_df = pd.read_csv("../municipality.csv")

**Select Only Active Students Without the Qualification**

In [9]:
#strip all columns
all_columns = ['Active / Not active/Early absorption','Name of Highest Qualification Achieved']
for column in all_columns:
    cohort_df[column] = cohort_df[column].astype(str)
    cohort_df[column] = cohort_df[column].str.strip()

cohort_df = cohort_df[cohort_df['Active / Not active/Early absorption'] == 'Active']
cohort_df = cohort_df[~cohort_df['Name of Highest Qualification Achieved'].str.contains('Comp')]

**Subset Datasets**

In [10]:
#subset emis and areacode dataset
national_df = national_df[['NatEmis', 'Suburb', 'Institution_Name']]
areacode_df = areacode_df[['STATSSA_Area_Code', 'Description']]

#subset cohort data
cohort_df = cohort_df[["Firstname","Surname", "ID Number/ Passport Number", "Birth Date", "Gender", "Disability", "Ethnicity", "Nationality", "Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)", "Home Language", "City", "Passed Grade 12 Y/N", "Municipality", "Name of High School", "Physical Address while studying", "Home Address", "Year Completed Grade 12"]]


**Drop NAN Rows**

In [11]:
#drop rows with null value in institution column
national_df = national_df[national_df["Institution_Name"].notna()]


**Create ID Column**

In [12]:
#create new idnumber column
cohort_df["IDNumber"] = [f"{id_num}".split("\\")[0].strip() for id_num in cohort_df["ID Number/ Passport Number"]]

#create new id type column
cohort_df["ID Type"] = ["National ID" if (len(f"{id_num}") == 13 or len(f"{id_num}") == 12) else "Passport Number" for id_num in cohort_df["IDNumber"]]

**Add Province**

In [13]:
#create second city column
cohort_df["City2"] = cohort_df["Home Address"].str.split(",").str[-2]

#convert second city column to type string
cohort_df["City2"] = cohort_df["City2"].astype(str)

#strip columns whitespaces
cohort_df["City2"] = cohort_df["City2"].str.strip()
province_df["City"] = province_df["City"].str.strip()

#set columns to lowercase
cohort_df["City2"] = cohort_df["City2"].str.lower()
province_df["City"] = province_df["City"].str.lower()

#prep search strings for fuzzywuzzy operation
strOptions = [elem for elem in province_df['City']]
strOptions = set(strOptions)
strOptions = list(strOptions)

#prep areas for fuzzywuzzy search
areas = [elem for elem in cohort_df["City2"]]
areas = set(areas)

#perform fuzzywuzzy matching
for area in areas:
    highest = process.extractOne(area, strOptions)
    cohort_df.loc[cohort_df['City2'] == area, 'Province'] = province_df.loc[province_df['City'] == highest[0], 'Province'].values[0]



**Split Physical Address**

In [14]:
#Split physical address into postal code
cohort_df["Physical Post Code"] = cohort_df["Physical Address while studying"].str.split(",").str[-1]

#Split physical address into city
cohort_df["Physical Address 3"] = cohort_df["Physical Address while studying"].str.split(",").str[-2]

#Split physical address into area
cohort_df["Physical Address 2"] = cohort_df["Physical Address while studying"].str.split(",").str[-3]

#Split physical address into street
cohort_df["Physical Address 1"] = cohort_df["Physical Address while studying"].str.split(",").str[0]

#Create physical province
cohort_df["Physical Province"] = cohort_df["Province"]

cohort_df[["Physical Address 1", "Physical Address 2", "Physical Address 3", "Physical Post Code", "Physical Province"]].head()

,Physical Address 1,Physical Address 2,Physical Address 3,Physical Post Code,Physical Province
1,66 8th Avenue,Bezuidenhout valley,Johannesburg,2094,Gauteng
2,84 Fox Street,Marshalltown,Johannesburg,2001,Gauteng
4,5456 Kgaka Cresent,Windmill Park Extension 12,Boksburg,1459,Gauteng
5,33 Kabul Crescent,Cosmo City,Randburg,2194,Gauteng
6,Glencairn Trafalgar,Corner Market & Eloff Street,Johannesburg,2000,KwaZulu-Natal


**Split Postal Address**

In [15]:
#Split physical address into postal code
cohort_df["Postal Post Code"] = cohort_df["Home Address"].str.split(",").str[-1]

#Split physical address into city
cohort_df["Postal Address 3"] = cohort_df["Home Address"].str.split(",").str[-2]

#Split physical address into area
cohort_df["Postal Address 2"] = cohort_df["Home Address"].str.split(",").str[-3]

#Split physical address into street
cohort_df["Postal Address 1"] = cohort_df["Home Address"].str.split(",").str[0]

#Create postal province
cohort_df["Postal Province"] = cohort_df["Province"]

cohort_df[["Postal Address 1", "Postal Address 2", "Postal Address 3", "Postal Post Code", "Postal Province"]].head()

,Postal Address 1,Postal Address 2,Postal Address 3,Postal Post Code,Postal Province
1,66 8th Avenue,Bezuidenhout Valley,Johannesburg,2094,Gauteng
2,84 Fox Street,Marshalltown,Johannesburg,2001,Gauteng
4,38144 Extension 18,Mnisi Avenue,Mamelodi East,0122,Gauteng
5,33 Kabul Crescent,Cosmo City,Randburg,2194,Gauteng
6,(46) B626 Asikhulume Lane,Durban,Umlazi,4066,KwaZulu-Natal


**Prepare EMIS and AREA Code Datasets**

In [16]:
#subset emis and areacode dataset
national_df = national_df[['NatEmis', 'Suburb', 'Institution_Name']]
areacode_df = areacode_df[['STATSSA_Area_Code', 'Description']]

#convert suburb and description to string
national_df['Suburb'] = national_df['Suburb'].astype(str)
national_df['Institution_Name'] = national_df['Institution_Name'].astype(str)
areacode_df['Description'] = areacode_df['Description'].astype(str)
cohort_df['Name of High School'] = cohort_df['Name of High School'].astype(str)

#Strip suburb and discription whitespaces
national_df['Suburb'] = national_df['Suburb'].str.strip()
areacode_df['Description'] = areacode_df['Description'].str.strip()

#Convert suburb and discription to lowercase
national_df['Suburb'] = national_df['Suburb'].str.lower()
areacode_df['Description'] = areacode_df['Description'].str.lower()



**Create EMIS column**

In [17]:
#prep search strings for fuzzywuzzy operation
strOptions = [elem for elem in national_df['Institution_Name']]
strOptions = set(strOptions)
strOptions = list(strOptions)

#prep areas for fuzzywuzzy search
areas = [elem for elem in cohort_df["Name of High School"]]
areas = set(areas)

#perform fuzzywuzzy matching
for area in areas:
    highest = process.extractOne(area, strOptions)
    cohort_df.loc[cohort_df['Name of High School'] == area, 'EMIS'] = national_df.loc[national_df['Institution_Name'] == highest[0], 'NatEmis'].values[0]
    cohort_df.loc[cohort_df['Name of High School'] == area, 'Area'] = national_df.loc[national_df['Institution_Name'] == highest[0], 'Suburb'].values[0]


    
cohort_df[["EMIS", "Area"]].head()

,EMIS,Area
1,700130641.0,nan
2,700220178.0,pretoria
4,700350371.0,nigel
5,930350040.0,extension
6,200501377.0,ntabankulu


**Format Emis Number**

In [18]:
cohort_df['EMIS'] = cohort_df['EMIS'].astype(int)
#eligible_df['EMIS'] = eligible_df['EMIS'].astype(str)

**Create AreaCode column**

In [19]:
#prep search strings for fuzzywuzzy operation
strOptions = [elem for elem in areacode_df['Description']]
strOptions = set(strOptions)
strOptions = list(strOptions)

#prep areas for fuzzywuzzy search
areas = [elem for elem in cohort_df["Area"]]
areas = set(areas)

#perform fuzzywuzzy matching
for area in areas:
    highest = process.extractOne(area, strOptions)
    cohort_df.loc[cohort_df['Area'] == area, 'AreaCode'] = areacode_df.loc[areacode_df['Description'] == highest[0], 'STATSSA_Area_Code'].values[0]
    
cohort_df[['EMIS', 'AreaCode']].head()
    

,EMIS,AreaCode
1,700130641,2011-962040001
2,700220178,2011-799035104
4,700350371,2011-797028016
5,930350040,2011-370004002
6,200501377,2011-298005001


**Reformat Municipality**

In [20]:
#convert second city column to type string
cohort_df["Municipality"] = cohort_df["Municipality"].astype(str)
municipality_df["municipality"] = municipality_df["municipality"].astype(str)

#strip columns whitespaces
cohort_df["Municipality"] = cohort_df["Municipality"].str.strip()
municipality_df["municipality"] = municipality_df["municipality"].str.strip()

#strip columns whitespaces
cohort_df["Municipality"] = cohort_df["Municipality"].str.lower()
municipality_df["municipality"] = municipality_df["municipality"].str.lower()

#prep search strings for fuzzywuzzy operation
strOptions = [elem for elem in municipality_df['municipality']]
strOptions = set(strOptions)
strOptions = list(strOptions)

#prep areas for fuzzywuzzy search
areas = [elem for elem in cohort_df["Municipality"]]
areas = set(areas)

#perform fuzzywuzzy matching
for area in areas:
    highest = process.extractOne(area, strOptions)
    cohort_df.loc[cohort_df['Municipality'] == area, 'municipality2'] = highest[0]

cohort_df[["Municipality", "municipality2"]].head()

,Municipality,municipality2
1,johannesburg metro,1709 johannesburg metro selwyn florida
2,johannesburg metro,1709 johannesburg metro selwyn florida
4,ekurhuleni metro,1519 ekurhuleni metro joy clinic daveyton
5,johannesburg metro,1709 johannesburg metro selwyn florida
6,johannesburg metro,1709 johannesburg metro selwyn florida


**Create New Columns**

In [21]:
#set marital status
cohort_df['MaritalStatus'] = "Single"

#set getc qualification
cohort_df['GETC Qualification'] = "GRADE 12"

#set popi act
cohort_df['Agree Popi Act'] = "Yes"

cohort_df[['MaritalStatus', 'GETC Qualification', 'Agree Popi Act']].head()

,MaritalStatus,GETC Qualification,Agree Popi Act
1,Single,GRADE 12,Yes
2,Single,GRADE 12,Yes
4,Single,GRADE 12,Yes
5,Single,GRADE 12,Yes
6,Single,GRADE 12,Yes


**Restructure Cohort Dataset**

In [22]:
#reorder and subset cohort dataset columns
cohort_df = cohort_df[['Firstname', 'Surname', 'IDNumber', 'ID Type', 'Birth Date', 'Gender', 'Disability', 
         'Ethnicity', 'Nationality',
       'Residential Status (Citizen / Permanent Resident/Asylum Seeker/Work Permit/Study Permit)',
       'Home Language', 'MaritalStatus', 'Province', 'GETC Qualification', 'municipality2', 'Physical Address 1', 'Physical Address 2',
       'Physical Address 3', 'Physical Post Code',  'Physical Province',
       'Postal Address 1', 'Postal Address 2', 'Postal Address 3',
       'Postal Post Code', 'Postal Province', 'EMIS', 'Year Completed Grade 12', 'AreaCode', 'Agree Popi Act',
       'Passed Grade 12 Y/N']]

#rename cohort dataset columns
cohort_df.columns = ['Name', 'Surname', 'IDNumber', 'ID Type', 'DateOfBirth(yyyy/mm/dd)', 'Gender',
                     'DisabilityStatus', 'Equity', 'Nationality', 'ResidentialStatus', 'Language',
                     'MaritalStatus', 'Province', 'GETC Qualification', 'Municipality', 'Physical Address 1',
                     'Physical Address 2', 'Physical  Address 3', 'Physical Post Code', 'Physical Province',
                     'Postal Address 1', 'Postal Address 2', 'Postal  Address 3', 'Postal Post Code',
                     'Postal Province', 'Emis Number', 'Last School Year', 'Area Code', 'Agree Popi Act',
                     'Grade 12']

cohort_df.head()

,Name,Surname,IDNumber,ID Type,DateOfBirth(yyyy/mm/dd),Gender,DisabilityStatus,Equity,Nationality,ResidentialStatus,...,Postal Address 1,Postal Address 2,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,Area Code,Agree Popi Act,Grade 12
1,Abigail,Hlalele,0005020136080,National ID,2000-05-02,Female,no,black,South African,Citizen,...,66 8th Avenue,Bezuidenhout Valley,Johannesburg,2094,Gauteng,700130641,2018,2011-962040001,Yes,Y
2,Arthur,Jenkins,0003255794087,National ID,2000-03-25,male,no,white,South African,Citizen,...,84 Fox Street,Marshalltown,Johannesburg,2001,Gauteng,700220178,-,2011-799035104,Yes,N
4,Majane,Thotse,9704115450080,National ID,1997-04-11,male,no,black,South African,Citizen,...,38144 Extension 18,Mnisi Avenue,Mamelodi East,0122,Gauteng,700350371,2015,2011-797028016,Yes,Y
5,Anza,Mugwabana,9406035830082,National ID,1994-06-03,male,no,black,South African,Citizen,...,33 Kabul Crescent,Cosmo City,Randburg,2194,Gauteng,930350040,2011,2011-370004002,Yes,Y
6,Andiswa,Nombela,9512140773081,National ID,1995-12-14,Female,no,black,South African,Citizen,...,(46) B626 Asikhulume Lane,Durban,Umlazi,4066,KwaZulu-Natal,200501377,2013,2011-298005001,Yes,Y


**Format Date of Biirth**

In [23]:
#convert dob to datetime formatt
cohort_df['DateOfBirth(yyyy/mm/dd)'] = pd.to_datetime(cohort_df['DateOfBirth(yyyy/mm/dd)'])

#convert dob cooulm to yy/mm/dd formatt
cohort_df['DateOfBirth(yyyy/mm/dd)'] = cohort_df['DateOfBirth(yyyy/mm/dd)'].dt.strftime('%Y/%d/%m')

cohort_df['DateOfBirth(yyyy/mm/dd)']

1      2000/02/05
2      2000/25/03
4      1997/11/04
5      1994/03/06
6      1995/14/12
          ...    
305    2000/13/11
307    1993/01/04
308    1991/15/08
309    1996/10/08
310    2001/05/08
Name: DateOfBirth(yyyy/mm/dd), Length: 258, dtype: object

**Format Language**

In [24]:
#strip equity column whitespaces
cohort_df['Language'] = cohort_df['Language'].str.strip()

#convert equity column to lowercase
cohort_df['Language'] = cohort_df['Language'].str.lower()

#reformat equity column
cohort_df['Language'] = cohort_df['Language'].replace({'isixhosa': 'isiXhosa', 'english': 'English',
                             'sepedi [also known as northern sotho / sesotho sa lebowa]': 'sePedi', 'tshivenda': 'tshiVenda',
                             'isizulu': 'isiZulu', 'setswana': 'seTswana', 'afrikaans': 'Afrikaans', 'xitsonga': 'xiTsonga',
                                'sepedi': 'sePedi', 'sesotho': 'seSotho', 'isindebele': 'isiNdebele','siswati': 'siSwati',  'other': 'English'})

cohort_df['Language'].unique()


array(['English', 'Afrikaans', 'sePedi', 'tshiVenda', 'isiZulu',
       'seTswana', 'xiTsonga', 'seSotho', 'isiNdebele', 'siSwati',
       'isiXhosa'], dtype=object)

**Format Equity**

In [25]:
#strip equity column whitespaces
cohort_df['Equity'] = cohort_df['Equity'].str.strip()

#convert equity column to lowercase
cohort_df['Equity'] = cohort_df['Equity'].str.lower()

#reformat equity column
cohort_df['Equity'] = cohort_df['Equity'].replace({'black': 'Black: African', 'coloured': 'Black: Coloured',
                             'indian': 'Black: Indian/Asian', 'asian': 'Black: Indian/Asian',
                             'chinese': 'Black: Indian/Asian', 'white': 'White'})

cohort_df['Equity'].unique()

array(['Black: African', 'White', 'Black: Indian/Asian',
       'Black: Coloured'], dtype=object)

**Format Disability**

In [26]:
#strip disability column whitespaces
cohort_df['DisabilityStatus'] = cohort_df['DisabilityStatus'].astype(str)

#strip disability column whitespaces
cohort_df['DisabilityStatus'] = cohort_df['DisabilityStatus'].str.strip()

#convert disability column to lowercase
cohort_df['DisabilityStatus'] = cohort_df['DisabilityStatus'].str.lower()

#reformat disability column
cohort_df['DisabilityStatus'] = cohort_df['DisabilityStatus'].replace({'no': 'None', 'nan': 'None',
                             'yes': 'Disable but unspecified', 'yes - specwearer': 'None',
                             'n': 'None', '-': 'None', 'visual impairment - spec wearer': 'None',
                             'aspergers / autistic': 'Emotional (behav/psych)', 'patella alta':'Physical (move/stand etc)',
                             'y- specwearer': 'None', 'add': 'None', 'visual impairment - spec wearer -': 'None',
                             '\\n': 'None', 'y - specwearer':'None', 'none':'None', '':'None'})

cohort_df['DisabilityStatus'].unique()

array(['None', 'Disable but unspecified', 'Emotional (behav/psych)',
       'Physical (move/stand etc)'], dtype=object)

**Format Province**

In [27]:
#strip equity column whitespaces
cohort_df['Province'] = cohort_df['Province'].str.strip()

#convert equity column to lowercase
cohort_df['Province'] = cohort_df['Province'].str.lower()

#reformat equity column
cohort_df['Province'] = cohort_df['Province'].replace({'gauteng': 'Gauteng', 'kwazulu-natal': 'Kwazulu-Natal',
                             'mpumalanga': 'Mpumalanga', 'eastern cape': 'Eastern Cape',
                             'limpopo': 'Limpopo', 'northern cape': 'Northern Cape', 'free state': 'Free State', 'north west': 'North West',
                                'western cape': 'Western Cape'})

cohort_df['Province'].unique()

array(['Gauteng', 'Kwazulu-Natal', 'Mpumalanga', 'Eastern Cape',
       'Limpopo', 'Northern Cape', 'Western Cape', 'Free State',
       'North West'], dtype=object)

**Format Nationality**

In [28]:
#strip equity column whitespaces
cohort_df['Nationality'] = cohort_df['Nationality'].str.strip()

#convert equity column to lowercase
cohort_df['Nationality'] = cohort_df['Nationality'].str.lower()

#reformat equity column
cohort_df['Nationality'] = cohort_df['Nationality'].replace({'south african': 'South African', 'dutch': 'European countries',
                             'nigerian': 'Rest of Africa', 'zimbabwean': 'Zimbabwe', 'congolese':'Rest of Africa'})

cohort_df['Nationality'].unique()

array(['South African', 'Zimbabwe', 'European countries',
       'Rest of Africa'], dtype=object)

**Format Physical Province**

In [29]:
#strip equity column whitespaces
cohort_df['Physical Province'] = cohort_df['Physical Province'].str.strip()

#convert equity column to lowercase
cohort_df['Physical Province'] = cohort_df['Physical Province'].str.lower()

#reformat equity column
cohort_df['Physical Province'] = cohort_df['Physical Province'].replace({'gauteng': 'Gauteng', 'kwazulu-natal': 'Kwazulu-Natal',
                             'mpumalanga': 'Mpumalanga', 'eastern cape': 'Eastern Cape',
                             'limpopo': 'Limpopo', 'northern cape': 'Northern Cape', 'free state': 'Free State', 'north west': 'North West',
                                'western cape': 'Western Cape'})

cohort_df['Physical Province'].unique()

array(['Gauteng', 'Kwazulu-Natal', 'Mpumalanga', 'Eastern Cape',
       'Limpopo', 'Northern Cape', 'Western Cape', 'Free State',
       'North West'], dtype=object)

**Format Postal Province**

In [30]:
#strip equity column whitespaces
cohort_df['Postal Province'] = cohort_df['Postal Province'].str.strip()

#convert equity column to lowercase
cohort_df['Postal Province'] = cohort_df['Postal Province'].str.lower()

#reformat equity column
cohort_df['Postal Province'] = cohort_df['Postal Province'].replace({'gauteng': 'Gauteng', 'kwazulu-natal': 'Kwazulu-Natal',
                             'mpumalanga': 'Mpumalanga', 'eastern cape': 'Eastern Cape',
                             'limpopo': 'Limpopo', 'northern cape': 'Northern Cape', 'free state': 'Free State', 'north west': 'North West',
                                'western cape': 'Western Cape'})

cohort_df['Postal Province'].unique()

array(['Gauteng', 'Kwazulu-Natal', 'Mpumalanga', 'Eastern Cape',
       'Limpopo', 'Northern Cape', 'Western Cape', 'Free State',
       'North West'], dtype=object)

**Split Into Eligible and Non-Eligble Datasets**

In [31]:
#incomplete datasets
#incomplete_df = cohort_df[cohort_df[['Name', 'Surname', 'IDNumber', 'ID Type', 'DateOfBirth(yyyy/mm/dd)',
                                    #'DisabilityStatus', 'Equity', 'Nationality', 'ResidentialStatus',
                                    #'Grade 12']].isnull().any(axis=1)]

#eligible dataset
#eligible_df = cohort_df.dropna(subset=['Name', 'Surname', 'IDNumber', 'ID Type', 'DateOfBirth(yyyy/mm/dd)',
                                    #'DisabilityStatus', 'Equity', 'Nationality', 'ResidentialStatus',
                                    #'Grade 12'])

#missing grade 12
#no_grade_12_df = eligible_df[eligible_df['Last School Year'].isin(['-'])]

#foreign grade 12
#foreign_grade_12_df = eligible_df[eligible_df['Grade 12'].isin(['Y - Foreign'])]
                             

IndentationError: unexpected indent (3108174842.py, line 3)

**Remove Incomplete Data from Eligible Dataset**

In [ ]:
#remove students without grade 12 from the eligibility list
#eligible_df = eligible_df[~eligible_df['IDNumber'].isin(no_grade_12_df['IDNumber'])]

#remove students with foreign grade 12 certs from eligible list
#eligible_df = eligible_df[~eligible_df['IDNumber'].isin(foreign_grade_12_df['IDNumber'])]

**Combine Ineligible datasets**

In [ ]:
#add issue column to incomplete dataset
#incomplete_df['issue'] = "Incomplete Data."

#add issue column to no grade 12 dataset
#no_grade_12_df['issue'] = "Did not complete grade 12."

#add issue column to foreign grade 12 dataset
#foreign_grade_12_df['issue'] = "Foreign grade 12 certificate."

#append dataframes
#not_eligible_df = incomplete_df.append(no_grade_12_df)
#not_eligible_df = not_eligible_df.append(foreign_grade_12_df)

#subset not eligible dataset
#not_eligible_df = not_eligible_df[['issue', 'Name', 'Surname', 'IDNumber', 'ID Type', 'DateOfBirth(yyyy/mm/dd)',
#                                    'DisabilityStatus', 'Equity', 'Nationality', 'ResidentialStatus',
#                                    'Grade 12']]

#not_eligible_df.head()

**Convert Postal Codes to Numeric**

In [32]:
eligible_df = cohort_df
#strip all columns
all_columns = ['Physical Post Code','Postal Post Code']
for column in all_columns:
    eligible_df[column] = eligible_df[column].astype(str)
    eligible_df[column] = eligible_df[column].str.strip()
    
    #update non numeric postal codes
    eligible_df.loc[~eligible_df[column].str.isnumeric(),column] = "2020"
    
#update non numeric postal codes
#eligible_df.loc[~eligible_df['Postal Post Code'].str.isnumeric(),'Postal Post Code'] = "2020"
#eligible_df.loc[~eligible_df['Physical Post Code'].str.isnumeric(),'Physical Post Code'] = "2020"

**Capitalize some Columns**

In [33]:
#strip all columns
all_columns = ['Gender','DisabilityStatus']
for column in all_columns:
    eligible_df[column] = eligible_df[column].astype(str)
    eligible_df[column] = eligible_df[column].str.strip()
    
    #capitalise columns
    eligible_df[column] = eligible_df[column].str.capitalize()
    

**Reformat ResidentialStatus**

In [34]:
#reformat disability column
eligible_df['ResidentialStatus'] = eligible_df['ResidentialStatus'].replace({'Citizen': 'South Africa'})
eligible_df['ResidentialStatus'].unique()

array(['South Africa', nan, 'Permanent Resident'], dtype=object)

**Capitalise Municipality**

In [35]:
#strip all columns
all_columns = ['Municipality']
for column in all_columns:
    eligible_df[column] = eligible_df[column].astype(str)
    eligible_df[column] = eligible_df[column].str.strip()
    
    #capitalise columns
    eligible_df[column] = eligible_df[column].str.title()

**Drop Some columns**

In [36]:
eligible_df = eligible_df[['Name', 'Surname', 'IDNumber', 'ID Type', 'DateOfBirth(yyyy/mm/dd)',
       'Gender', 'DisabilityStatus', 'Equity', 'Nationality',
       'ResidentialStatus', 'Language', 'MaritalStatus', 'Province',
       'GETC Qualification', 'Municipality', 'Physical Address 1',
       'Physical Address 2', 'Physical  Address 3', 'Physical Post Code',
       'Physical Province', 'Postal Address 1', 'Postal Address 2',
       'Postal  Address 3', 'Postal Post Code', 'Postal Province',
       'Emis Number', 'Last School Year', 'Area Code', 'Agree Popi Act']]

**Write Datasets to Excel file**

In [38]:
#write eligible dataset to eligible file
eligible_df.to_excel("../seta_eligible_second_upload.xlsx", index=False)

#write ineligible dataset to ineligible file
#not_eligible_df.to_excel("../ineligible.xlsx")

In [39]:
eligible_df = pd.read_excel("../seta_eligible_second_upload.xlsx")
eligible_df['Name'] = eligible_df['Name'].astype(str).str.strip()
eligible_df['Surname'] = eligible_df['Surname'].astype(str).str.strip()

In [133]:
#condition_name = eligible_df['IDNumber'] == 9604165085085
condition_name = eligible_df['Name'] == "Abdullah"

eligible_df[condition_name]

,Name,Surname,IDNumber,ID Type,DateOfBirth(yyyy/mm/dd),Gender,DisabilityStatus,Equity,Nationality,ResidentialStatus,...,Physical Province,Postal Address 1,Postal Address 2,Postal Address 3,Postal Post Code,Postal Province,Emis Number,Last School Year,Area Code,Agree Popi Act
171,Abdullah,Ebrahim,0001165311083,National ID,2000/16/01,Male,None,Black: Coloured,South African,South Africa,...,Gauteng,65 Stockley Road,Kenwyn,Lansdowne,7780,Gauteng,100000636,2020,2011-199041056,Yes


In [134]:
for column in eligible_df.columns:
    print(f"{column} : {eligible_df[condition_name][column].values[0]}")

Name : Abdullah
Surname : Ebrahim
IDNumber : 0001165311083
ID Type : National ID
DateOfBirth(yyyy/mm/dd) : 2000/16/01
Gender : Male
DisabilityStatus : None
Equity : Black: Coloured
Nationality : South African
ResidentialStatus : South Africa
Language : English
MaritalStatus : Single
Province : Gauteng
GETC Qualification : GRADE 12
Municipality : 7570 Cape Town Metro Arcadia Centre Kraaifontein
Physical Address 1 : 65 Stockley Road
Physical Address 2 :  Kenwyn
Physical  Address 3 :  Lansdowne
Physical Post Code : 7780
Physical Province : Gauteng
Postal Address 1 : 65 Stockley Road
Postal Address 2 :  Kenwyn
Postal  Address 3 :  Lansdowne
Postal Post Code : 7780
Postal Province : Gauteng
Emis Number : 100000636
Last School Year : 2020
Area Code : 2011-199041056
Agree Popi Act : Yes


In [2]:
eligible_df = pd.read_excel("../seta_eligible_second_upload.xlsx")
ineligible_df = pd.read_excel("../ineligible.xlsx")

In [8]:
ineligible_df.head()

,Unnamed: 0,issue,Name,Surname,IDNumber,ID Type,DateOfBirth(yyyy/mm/dd),DisabilityStatus,Equity,Nationality,ResidentialStatus,Grade 12
0,37,Incomplete Data.,Kalombo,Bondo,9109209087188,National ID,1991/20/09,None,Black: African,Rest of Africa,NaN,Y
1,84,Incomplete Data.,Lloyd,Ndhlovu,CN806411,Passport Number,1992/23/02,None,Black: African,Zimbabwe,NaN,Y
2,217,Incomplete Data.,Bryan,Nkala,9612196335189,National ID,1996/19/12,None,Black: African,Zimbabwe,NaN,Y
3,220,Incomplete Data.,Charney,Essack,9909210274083,National ID,1999/21/09,None,Black: Coloured,South African,NaN,Y
4,267,Incomplete Data.,Mesuli,Ndzawumbi,9104275767086,National ID,1991/27/04,None,Black: African,South African,NaN,Y
